In [6]:
import pandas as pd


ds_json = pd.read_json('data/lorawan_antwerp_2019_dataset.json')
gw_loc = pd.read_json('data/lorawan_antwerp_gateway_locations.json')

In [7]:
ds_json

,adr,hdop,counter,longitude,sf,airtime,dev_addr,gateways,latitude,dev_eui,payload,channel
0,1,0.66,100086,4.427051,7,0.112896,07000EFF,"[{'id': '080605EE', 'rssi': -104, 'snr': -3.25...",51.235508,343233386A377C18,00eb000329f14c4267aa8d40c3f5283f0400ca55410510...,6
1,1,0.83,62377,4.397273,7,0.112896,07000EAD,"[{'id': '08060716', 'rssi': -99, 'snr': 10.0, ...",51.203171,3432333855378A18,003900030cd04c4276b68c40e17a543f048051bf4105c0...,8
2,1,0.70,31952,4.400576,7,0.112896,07000EFE,"[{'id': '08060716', 'rssi': -115, 'snr': 6.75,...",51.208565,3432333853376B18,008d000392d54c4284d18c403333333f04682aa9410500...,8
3,1,0.64,57377,4.391779,7,0.112896,07000FB2,"[{'id': 'FF0178DF', 'rssi': -85, 'snr': -5.0, ...",51.202320,3432333864377C18,00c300032dcf4c4275898c400bd7233f0430896b410540...,3
4,1,0.64,112480,4.417615,8,0.205312,07000ECB,"[{'id': 'FF0107C9', 'rssi': -114, 'snr': -4.0,...",51.211182,3432333855376418,00f6000340d84c421a5d8d400bd7233f040832bb4105a0...,7
...,...,...,...,...,...,...,...,...,...,...,...,...
130425,1,0.67,84868,4.409659,7,0.112896,07000FB2,"[{'id': 'FF0107C9', 'rssi': -110, 'snr': 3.0, ...",51.220184,3432333864377C18,001c000378e14c42ed1b8d401f852b3f04909688410570...,5
130426,1,0.72,35,4.418367,7,0.112896,07000EFD,"[{'id': '080E00B9', 'rssi': -103, 'snr': 10.25...",51.194546,3432333860376818,0021000337c74c4244638d40eb51383f04e8be0f420540...,8
130427,1,0.72,84867,4.409659,7,0.112896,07000FB2,"[{'id': 'FF0107C9', 'rssi': -105, 'snr': 1.0, ...",51.220184,3432333864377C18,001b000378e14c42ee1b8d40eb51383f046059874105b0...,3
130428,1,0.73,88640,4.409161,7,0.112896,07000EAD,"[{'id': 'FF010560', 'rssi': -114, 'snr': -8.0,...",51.233089,3432333855378A18,00d30003afee4c42d8178d4047e13a3f04d83baa410520...,1


In [10]:
gw_loc.T['FF010F35']['latitude']

50.898949

In [24]:
gw_meta = ds_json.latitude

In [25]:
gw_meta[1]

51.20317077636719

In [29]:
# Loading initial position coordinates form machine learning predictions
pos_pred_rssi = pd.read_csv('files/position_pred_RSSI.csv', index_col=0)
pos_pred_comb = pd.read_csv('files/position_pred_weather-comb.csv', index_col=0)

import pymap3d as pm 

pos_pred_rssi['x'], pos_pred_rssi['y'], _ = pm.geodetic2ecef(lat=pos_pred_rssi['lat'], lon=pos_pred_rssi['lon'], alt=0)
pos_pred_rssi['x_i'], pos_pred_rssi['y_i'], _ = pm.geodetic2ecef(lat=pos_pred_rssi['pred_lat_rssi'], lon=pos_pred_rssi['pred_lon_rssi'], alt=0)

pos_pred_comb['x'], pos_pred_comb['y'], _ = pm.geodetic2ecef(lat=pos_pred_comb['lat'], lon=pos_pred_comb['lon'], alt=0)
pos_pred_comb['x_i'], pos_pred_comb['y_i'], _ = pm.geodetic2ecef(lat=pos_pred_comb['pred_lat_rssi'], lon=pos_pred_comb['pred_lon_rssi'], alt=0)


The idea is to accumulate the actual lat-lon data, predicted ones, the converted lat-lon values, including the lat-lon vlaues for initialization (these initial values are the converted location values predicted by the machine learning model) into a single dataframe and file.

Later
1. we will pick each sample from this dataset, 
2. take the actual lat-lon values from the sample 
3. Find the corresponding data point from the ds_json raw data collection 
4. Get the corresponding GW metadata from the Gateways column
5. collect the TOA values for each GW and their GWID
6. We will get the lat-lon of the corresponding GW in the gw_loc df and convert it to ecef 
7. Do the position estimation
8. Compare the ecef coordinates with the actual values 
9. Finally, convert to the geodetic coordinate and calculate the defference/error '   


In [ ]:
pos_pred_rssi.to_csv('files/pos_pred_rssi_ecef.csv')
pos_pred_comb.to_csv('files/pos_pred_comb_ecef.csv')